In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [3]:
model_name="deepseek-r1-distill-llama-70b"

In [4]:
from langchain_groq import ChatGroq

In [5]:
llm=ChatGroq(model_name=model_name)

In [6]:
print(llm.invoke("What is your name?").content)

<think>

</think>

Greetings! I'm DeepSeek-R1, an artificial intelligence assistant created by DeepSeek. I'm at your service and would be delighted to assist you with any inquiries or tasks you may have.


In [11]:
import operator
from typing import List
from pydantic import BaseModel,Field
from typing import TypedDict,Annotated,Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.messages import AIMessage,HumanMessage
from langgraph.graph import Graph